In [1]:
import os
import pandas as pd

# Pedir al usuario la carpeta raíz
carpeta_raiz = input("Ingresa la ruta de la carpeta raíz: ").strip()

# Buscar el único archivo Excel en la carpeta raíz
excel_files = [f for f in os.listdir(carpeta_raiz) if f.endswith('.xlsx') or f.endswith('.xls')]

if len(excel_files) != 1:
    print("⚠️ No se encontró exactamente un archivo Excel en la carpeta raíz.")
    exit()

# Cargar el archivo Excel encontrado
ruta_excel = os.path.join(carpeta_raiz, excel_files[0])
df = pd.read_excel(ruta_excel)

# Asegurar que las columnas estén bien nombradas
df.columns = [col.strip().upper() for col in df.columns]
df = df.rename(columns={
    "DOCUMENTO": "DOCUMENTO",
    "ASEGURADORA": "ASEGURADORA",
    "PROGRAMA": "PROGRAMA",
    "APELLIDOS Y NOMBRES": "NOMBRE_COMPLETO"
})

# Crear columna con el nombre del paciente en formato para búsqueda
df["NOMBRE_BUSQUEDA"] = df["NOMBRE_COMPLETO"].str.upper().str.replace(" ", "", regex=False)

# Obtener subcarpetas dentro de la carpeta raíz (cada una representa un tipo de soporte)
subcarpetas = [carpeta for carpeta in os.listdir(carpeta_raiz)
               if os.path.isdir(os.path.join(carpeta_raiz, carpeta))]

# Crear columnas nuevas por cada subcarpeta y llenarlas con ""
for sub in subcarpetas:
    df[sub] = ""

# Buscar archivos PDF por subcarpeta
for sub in subcarpetas:
    ruta_sub = os.path.join(carpeta_raiz, sub)
    archivos = os.listdir(ruta_sub)
    archivos_pdf = [f for f in archivos if f.lower().endswith('.pdf')]
    
    for idx, fila in df.iterrows():
        nombre_paciente = fila["NOMBRE_BUSQUEDA"]
        encontrado = any(nombre_paciente in archivo.replace(" ", "").upper() for archivo in archivos_pdf)
        if encontrado:
            df.at[idx, sub] = "OK"

# Eliminar la columna auxiliar
df.drop(columns=["NOMBRE_BUSQUEDA"], inplace=True)

# Guardar nuevo archivo Excel
archivo_salida = os.path.join(carpeta_raiz, "reporte_completo.xlsx")
df.to_excel(archivo_salida, index=False)

print("✅ Archivo generado con éxito:", archivo_salida)


✅ Archivo generado con éxito: C:\Users\AUX ADMINISTRATIVO\Documents\TOBAR\BOOTCAMP PROGRAMACION\Phyton proyecto\07 JULIO\reporte_completo.xlsx
